<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting-up-HRM" data-toc-modified-id="Setting-up-HRM-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting up HRM</a></span><ul class="toc-item"><li><span><a href="#Model-of-mapping-genes-to-0" data-toc-modified-id="Model-of-mapping-genes-to-0-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Model of mapping genes to 0</a></span></li><li><span><a href="#Model-of-Dropping-Genes" data-toc-modified-id="Model-of-Dropping-Genes-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model of Dropping Genes</a></span></li></ul></li><li><span><a href="#Validation" data-toc-modified-id="Validation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Validation</a></span><ul class="toc-item"><li><span><a href="#Validation-of-dropping-genes" data-toc-modified-id="Validation-of-dropping-genes-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Validation of dropping genes</a></span></li></ul></li><li><span><a href="#Validation-of-Mapping-Genes-to-0" data-toc-modified-id="Validation-of-Mapping-Genes-to-0-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Validation of Mapping Genes to 0</a></span></li><li><span><a href="#START-INCORPORATING-NEW-DEEP-DIVES-HERE!" data-toc-modified-id="START-INCORPORATING-NEW-DEEP-DIVES-HERE!-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>START INCORPORATING NEW DEEP DIVES HERE!</a></span></li></ul></div>

In [1]:
import pandas as pd
from pathlib import Path
from cdm_src.host_response_model import HostResponseModel
from harness.th_model_instances.hamed_models.random_forest_regression import random_forest_regression
from harness.th_model_instances.hamed_models.rocklin_models import linear_regression, gradboost_regression
import os

In [3]:
#Setup directory
dir_path = '.'
if not os.path.exists(dir_path):
    print('creating path')
    os.makedirs(dir_path)


In [4]:

hrm_target_col = "logFC"
hrm_gene_col = "gene"


In [6]:
# Read in dataframe

hrm_data = pd.read_csv('./ecoli_additive_design_df.csv')
hrm_data.rename(columns={"Unnamed: 0": "gene"}, inplace=True)
hrm_data['gene']=hrm_data['gene'].apply(str.lower)
print(hrm_data.shape)
print(hrm_data.shape)
print("\nHRM data looks like:")
hrm_data.head(4)

(61665, 9)
(61665, 9)

HRM data looks like:


,gene,flagedgeRremoved_MG1655_WT,FDR,nlogFDR,logFC,IPTG_concentration,arabinose_concentration,timepoint,strain
0,actuator_yfp,1,0.0,0.0,0.0,1,0,5.0,MG1655_WT
1,camr,1,0.0,0.0,0.0,1,0,5.0,MG1655_WT
2,circuit_icar,1,0.0,0.0,0.0,1,0,5.0,MG1655_WT
3,circuit_phlf,1,0.0,0.0,0.0,1,0,5.0,MG1655_WT


In [7]:
hrm_data = hrm_data.merge(pd.get_dummies(hrm_data['timepoint'],'timepoint'),left_index=True,right_index=True)
hrm_experimental_condition_cols=['IPTG_concentration','arabinose_concentration','timepoint_5.0','timepoint_6.5',
                                'timepoint_8.0','timepoint_18.0']

In [8]:
#Read in translated network. Translation here refers to Ecolinet with gene symbols instead of locus tags.
org_network = pd.read_csv('./CX.INT.EcoliNet.v1_translated.csv')

# Setting up HRM

In [9]:
print(dir_path)

.


In [10]:
hrm = HostResponseModel(initial_data=hrm_data, output_path=dir_path, leaderboard_query=None,
                        exp_condition_cols=hrm_experimental_condition_cols, target_col=hrm_target_col,
                        gene_col=hrm_gene_col)


Column IPTG_concentration: contains 2 unique values
Column arabinose_concentration: contains 2 unique values
Column timepoint_5.0: contains 2 unique values
Column timepoint_6.5: contains 2 unique values
Column timepoint_8.0: contains 2 unique values
Column timepoint_18.0: contains 2 unique values
Input dataframe contains 15 conditions out of 64 possible conditions
There are 49 conditions to be predicted



In [12]:
#Embed the prior network for the model
print(hrm.output_path)
hrm.existing_data['gene']=hrm.existing_data['gene'].apply(str.lower)
hrm.embed_prior_network(df_network = org_network)


./cdm_outputs


Computing transition probabilities:   0%|          | 2/4039 [00:00<04:01, 16.71it/s]

Building model...


Computing transition probabilities: 100%|██████████| 4039/4039 [00:33<00:00, 119.72it/s]



Fitting model...
Embedding columns were added in self.existing_data
Embedding columns were added in self.future_data



In [14]:
#Confirm that number of genes has not changed
len(hrm.existing_data['gene'].unique())

4111

In [15]:
#Check that the number of genes with an embedding is constant across all the conditions
hrm.existing_data.groupby(['IPTG_concentration', 'arabinose_concentration', 'timepoint', 'strain', 'timepoint_5.0', 'timepoint_6.5', 'timepoint_8.0', 'timepoint_18.0']).sum()['emb_present']

IPTG_concentration  arabinose_concentration  timepoint  strain     timepoint_5.0  timepoint_6.5  timepoint_8.0  timepoint_18.0
0                   0                        6.5        MG1655_WT  0              1              0              0                 3818.0
                                             8.0        MG1655_WT  0              0              1              0                 3818.0
                                             18.0       MG1655_WT  0              0              0              1                 3818.0
                    1                        5.0        MG1655_WT  1              0              0              0                 3818.0
                                             6.5        MG1655_WT  0              1              0              0                 3818.0
                                             8.0        MG1655_WT  0              0              1              0                 3818.0
                                             18.0  

In [16]:
#Create train/test splits. No validation set here, as this is just a proof of concept.
train_df = hrm.existing_data[~(((hrm.existing_data['IPTG_concentration']==1)&
                                (hrm.existing_data['arabinose_concentration']==1)))].fillna(0)
test_df = hrm.existing_data[(((hrm.existing_data['IPTG_concentration']==1)&
                                (hrm.existing_data['arabinose_concentration']==1)))].fillna(0)
print(len(train_df),len(test_df))

45221 16444


## Model of mapping genes to 0

In [23]:
print("\n--------------------------------- starting HRM model testing - mapping genes to 0 ---------------------------------\n")

th_kwargs = dict(function_that_returns_TH_model=random_forest_regression,
                 dict_of_function_parameters={},
                 description="embedding_all_genes",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data.fillna(0), percent_train='NA',num_pred_conditions=2,**th_kwargs)


th_kwargs = dict(function_that_returns_TH_model=linear_regression,
                 dict_of_function_parameters={},
                 description="embedding_all_genes",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data.fillna(0), percent_train='NA',num_pred_conditions=2,**th_kwargs)

th_kwargs = dict(function_that_returns_TH_model=gradboost_regression,
                 dict_of_function_parameters={},
                 description="embedding_all_genes",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data.fillna(0), percent_train='NA',num_pred_conditions=2,**th_kwargs)



--------------------------------- starting HRM model testing - mapping genes to 0 ---------------------------------




/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


----------------------------------------------------------------------------------------------------
Starting run of model random_forest_regression at time 12:31:52
Starting Regression training...
Training time was: 33.25 seconds
Testing time was: 0.94 seconds
Prediction time of untested data was: 3.99837589263916
Run finished at 12:32:38. Total run time = 45.80 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


----------------------------------------------------------------------------------------------------
Starting run of model linear_regression at time 12:32:38
Starting Regression training...



/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


Training time was: 0.05 seconds
Testing time was: 0.02 seconds
Prediction time of untested data was: 0.2449331283569336
Run finished at 12:32:46. Total run time = 8.04 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


----------------------------------------------------------------------------------------------------
Starting run of model gradboost_regression at time 12:32:46



/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


Starting Regression training...
Training time was: 64.07 seconds
Testing time was: 0.05 seconds
Prediction time of untested data was: 0.9202649593353271
Run finished at 12:33:59. Total run time = 72.47 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^




## Model of Dropping Genes

In [24]:
print("\n--------------------------------- starting HRM model testing - dropping genes not in emb ---------------------------------\n")

train_df = hrm.existing_data[(~(((hrm.existing_data['IPTG_concentration']==1)&
                                (hrm.existing_data['arabinose_concentration']==1))))&\
                            (hrm.existing_data['emb_present']==1)]
test_df = hrm.existing_data[(((hrm.existing_data['IPTG_concentration']==1)&
                                (hrm.existing_data['arabinose_concentration']==1)))&\
                            (hrm.existing_data['emb_present']==1)]
print(len(train_df),len(test_df))

th_kwargs = dict(function_that_returns_TH_model=random_forest_regression,
                 dict_of_function_parameters={},
                 description="embedding_network_genes_only",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data[hrm.future_data['emb_present']==1], percent_train='NA',num_pred_conditions=2,**th_kwargs)


th_kwargs = dict(function_that_returns_TH_model=linear_regression,
                 dict_of_function_parameters={},
                 description="embedding_network_genes_only",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data[hrm.future_data['emb_present']==1], percent_train='NA',num_pred_conditions=2,**th_kwargs)

th_kwargs = dict(function_that_returns_TH_model=gradboost_regression,
                 dict_of_function_parameters={},
                 description="embedding_network_genes_only",
                 feature_cols_to_use=hrm_experimental_condition_cols+['embcol_'+str(i) for i in range(32)],
                 feature_cols_to_normalize=['embcol_'+str(i) for i in range(32)])



hrm._invoke_test_harness(train_df, test_df, hrm.future_data[hrm.future_data['emb_present']==1], percent_train='NA',num_pred_conditions=2,**th_kwargs)


--------------------------------- starting HRM model testing - dropping genes not in emb ---------------------------------

41998 15272
----------------------------------------------------------------------------------------------------
Starting run of model random_forest_regression at time 12:36:23
Starting Regression training...



/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


Training time was: 28.68 seconds
Testing time was: 1.03 seconds
Prediction time of untested data was: 0.4291110038757324
Run finished at 12:36:56. Total run time = 33.01 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


----------------------------------------------------------------------------------------------------
Starting run of model linear_regression at time 12:36:56
Starting Regression training...
Training time was: 0.03 seconds
Testing time was: 0.01 seconds



/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


Prediction time of untested data was: 0.02236199378967285
Run finished at 12:36:59. Total run time = 3.05 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


----------------------------------------------------------------------------------------------------
Starting run of model gradboost_regression at time 12:36:59
Starting Regression training...



/Users/meslami/Documents/GitRepos/cdm/cdm_src/cdm_base_class.py:138:
You are overwriting the features to use, this may impact downstream integration with predictions....


Training time was: 62.08 seconds
Testing time was: 0.05 seconds
Prediction time of untested data was: 0.03720808029174805
Run finished at 12:38:04. Total run time = 64.76 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^




In [30]:
hrm.existing_data.to_csv(os.path.join(dir_path,'hrmdata.csv'))
hrm.existing_data['emb_present'].value_counts()

1.0    57270
0.0     4395
Name: emb_present, dtype: int64

In [31]:
train_df.columns

Index(['gene', 'flagedgeRremoved_MG1655_WT', 'FDR', 'nlogFDR', 'logFC', 'IPTG_concentration', 'arabinose_concentration', 'timepoint', 'strain', 'timepoint_5.0', 'timepoint_6.5', 'timepoint_8.0', 'timepoint_18.0', 'embcol_0', 'embcol_1', 'embcol_2', 'embcol_3', 'embcol_4', 'embcol_5', 'embcol_6', 'embcol_7', 'embcol_8', 'embcol_9', 'embcol_10', 'embcol_11', 'embcol_12', 'embcol_13', 'embcol_14', 'embcol_15', 'embcol_16', 'embcol_17', 'embcol_18', 'embcol_19', 'embcol_20', 'embcol_21', 'embcol_22', 'embcol_23', 'embcol_24', 'embcol_25', 'embcol_26', 'embcol_27', 'embcol_28', 'embcol_29', 'embcol_30', 'embcol_31', 'emb_present'], dtype='object')

In [33]:
# Note, this will take a long time.
th_kwargs = dict(function_that_returns_TH_model=random_forest_regression,
                 dict_of_function_parameters={},
                 description="sparse_genes",
                 sparse_cols_to_use=['gene'])

hrm._invoke_test_harness(train_df, test_df, test_df, percent_train='NA',num_pred_conditions=2,**th_kwargs)


----------------------------------------------------------------------------------------------------
Starting run of model random_forest_regression at time 16:55:21
Starting Regression training...
Training time was: 868.90 seconds
Testing time was: 1.29 seconds
Prediction time of untested data was: 1.2144639492034912
Run finished at 17:09:56. Total run time = 874.91 seconds
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^




In [ ]:
# Check leaderboard and subfolders in cdm_outputs for all outputs.